# Working Iceberg SQL Magic Example

This notebook demonstrates how to use **%sparksql** magic with Iceberg tables in your LASAGNA setup.

## ✅ What Works:
- **%sparksql** magic with Iceberg tables
- Interactive table outputs
- Complex SQL queries
- Cross-database queries

## ❌ What Doesn't Work:
- DataFrame API with `spark.read.format("iceberg")` (due to metadata issues)
- Direct file path access to Iceberg tables


## 1. Create Spark Session with Iceberg Support


## 2. Run Ingestion Script (Optional)


In [ ]:
# Run the ingestion script to create tables and data
# This is optional - tables may already exist from previous runs
import subprocess
import sys

try:
    result = subprocess.run([
        sys.executable, 
        "/usr/local/spark_dev/work/payments_pipeline_ingestion/src/payments_pipeline/bronze/ingestion.py"
    ], capture_output=True, text=True, cwd="/usr/local/spark_dev/work")
    
    if result.returncode == 0:
        print("✅ Ingestion script completed successfully")
        print("Tables and data are ready for SQL magic queries")
    else:
        print("⚠️  Ingestion script had issues, but tables might already exist")
        print("Proceeding with existing tables...")
        
except Exception as e:
    print(f"⚠️  Could not run ingestion script: {e}")
    print("Proceeding with existing tables...")


In [4]:
from pyspark.sql import SparkSession

# Stop any existing session
try:
    spark.stop()
except:
    pass

# Create Spark session with Iceberg support
spark = SparkSession.builder \
    .appName("LASAGNA-Iceberg-SQL") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.4.2") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .getOrCreate()

print("✅ Spark session created with Iceberg support")


25/09/29 16:41:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/09/29 16:41:52 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


✅ Spark session created with Iceberg support


## 3. Basic Iceberg Table Queries


In [5]:
%%sparksql
-- Count records in merchants_raw table
SELECT COUNT(*) as total_merchants 
FROM spark_catalog.payments_bronze.merchants_raw


[Stage 3:>                                                          (0 + 2) / 2]

Execution time: 5.00 seconds


SparkSchemaWidget(nodes=(Node(close_icon='angle-down', close_icon_style='danger', icon='project-diagram', icon…

total_merchants
1000


In [7]:
%%sparksql
-- Sample data from merchants_raw
SELECT 
    merchant_id,
    merchant_name,
    industry,
    city,
    state,
    mdr_rate
FROM spark_catalog.payments_bronze.merchants_raw 


Execution time: 0.45 seconds
Only showing top 20 rows


SparkSchemaWidget(nodes=(Node(close_icon='angle-down', close_icon_style='danger', icon='project-diagram', icon…

merchant_id,merchant_name,industry,city,state,mdr_rate
M000001,Prime Center 628,automotive,Chicago,IL,0.0285
M000002,Elite Center 983,automotive,New York,AZ,0.026
M000003,Prime Center 717,technology,Chicago,AZ,0.0208
M000004,Best Mart 97,automotive,Philadelphia,AZ,0.0277
M000005,Prime Plaza 252,technology,Phoenix,AZ,0.0235
M000006,Super Mart 216,local_services,Houston,PA,0.0318
M000007,Super Plaza 264,fitness,Chicago,CA,0.0306
M000008,Prime Store 442,retail,Chicago,TX,0.0283
M000009,Mega Mart 402,restaurant,Chicago,CA,0.0307
M000010,Super Center 407,beauty,Chicago,IL,0.029


## 3. Complex Analytics Queries


In [4]:
%%sparksql
-- Industry analysis
SELECT 
    industry,
    COUNT(*) as merchant_count,
    ROUND(AVG(mdr_rate), 4) as avg_mdr_rate,
    ROUND(MIN(mdr_rate), 4) as min_mdr_rate,
    ROUND(MAX(mdr_rate), 4) as max_mdr_rate
FROM spark_catalog.payments_bronze.merchants_raw 
GROUP BY industry 
ORDER BY merchant_count DESC


Execution time: 0.89 seconds


SparkSchemaWidget(nodes=(Node(close_icon='angle-down', close_icon_style='danger', icon='project-diagram', icon…

industry,merchant_count,avg_mdr_rate,min_mdr_rate,max_mdr_rate
retail,228,0.0305,0.0201,0.035
restaurant,184,0.0306,0.0251,0.0349
fitness,143,0.0321,0.029,0.0349
local_services,135,0.0319,0.0291,0.0349
beauty,131,0.032,0.029,0.035
healthcare,57,0.0261,0.0204,0.029
automotive,51,0.027,0.025,0.029
education,46,0.0271,0.0251,0.029
technology,10,0.0223,0.0201,0.0246
logistics,9,0.0224,0.0206,0.0235


In [5]:
%%sparksql
-- Geographic distribution
SELECT 
    state,
    COUNT(*) as merchant_count,
    ROUND(AVG(mdr_rate), 4) as avg_mdr_rate
FROM spark_catalog.payments_bronze.merchants_raw 
GROUP BY state 
ORDER BY merchant_count DESC 
LIMIT 10


Execution time: 0.55 seconds


SparkSchemaWidget(nodes=(Node(close_icon='angle-down', close_icon_style='danger', icon='project-diagram', icon…

state,merchant_count,avg_mdr_rate
AZ,184,0.0302
NY,172,0.0304
PA,165,0.0306
IL,164,0.0301
CA,162,0.0305
TX,153,0.0303


In [9]:
%%sparksql
SELECT 
    merchant_name,
    industry,
    COUNT(*) as transaction_count,
    SUM(payment_amount) as total_amount
FROM spark_catalog.payments_bronze.merchants_raw m
JOIN spark_catalog.payments_bronze.transactions_raw t ON m.merchant_id = t.merchant_id
GROUP BY merchant_name, industry
ORDER BY transaction_count DESC
LIMIT 10

Execution time: 2.78 seconds


SparkSchemaWidget(nodes=(Node(close_icon='angle-down', close_icon_style='danger', icon='project-diagram', icon…

merchant_name,industry,transaction_count,total_amount
Prime Mart 421,manufacturing,5100,2790039.209999999
Prime Store 335,healthcare,4762,2627184.200000002
Elite Mart 409,technology,4455,2460667.990000001
Best Mart 741,retail,4343,2385646.709999998
Mega Store 108,logistics,4337,2373022.8000000017
Mega Mart 499,healthcare,4258,2314130.709999997
Mega Plaza 88,healthcare,4225,2334957.5999999996
Best Mart 984,healthcare,4217,2318210.6799999983
Prime Store 101,logistics,4214,2324500.25
Prime Store 269,healthcare,4179,2290924.51


## 5. Troubleshooting Guide

### ✅ What Works:
- **%sparksql** magic with full catalog path: `spark_catalog.payments_bronze.merchants_raw`
- Interactive table outputs with automatic pagination
- Complex SQL queries (JOINs, window functions, aggregations)
- Cross-database queries

### ❌ What Doesn't Work:
- DataFrame API: `spark.read.format("iceberg").load("s3a://...")`
- Direct file path access to Iceberg tables
- Trino magic with Iceberg tables (catalog issues)

### 🔧 Key Requirements:
1. **Use full catalog path**: `spark_catalog.payments_bronze.merchants_raw`
2. **Include Iceberg packages** in Spark session
3. **Configure S3A** for MinIO access
4. **Use %sparksql magic** instead of DataFrame API

### 🚀 Performance Tips:
- Use `LIMIT` for exploration queries
- Select specific columns instead of `SELECT *`
- Use proper WHERE clauses to filter data
- Consider partitioning for large tables
